In [381]:
# this implementation employs ideas from
# https://people.eecs.berkeley.edu/~nagaban2/resources/portfolio/convex_report.pdf


In [382]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [383]:
x = torch.rand(3)
x

tensor([0.418, 0.161, 0.196])

In [384]:
y = torch.log(x)
y

tensor([-0.872, -1.829, -1.632])

In [385]:
w = torch.rand(3, 4, requires_grad=True)
w

tensor([[0.787, 0.835, 0.135, 0.393],
        [0.275, 0.923, 0.220, 0.356],
        [0.526, 0.985, 0.216, 0.635]], requires_grad=True)

In [386]:
b = torch.rand(4, requires_grad=True)
b

tensor([0.860, 0.025, 0.069, 0.203], requires_grad=True)

In [387]:
z = [torch.matmul(w[:,i], x) + b[i] for i in range(4)]
z

[tensor(1.336, grad_fn=<AddBackward0>),
 tensor(0.715, grad_fn=<AddBackward0>),
 tensor(0.203, grad_fn=<AddBackward0>),
 tensor(0.548, grad_fn=<AddBackward0>)]

In [388]:
zh = [torch.exp(zz) for zz in z]
zh

[tensor(3.804, grad_fn=<ExpBackward>),
 tensor(2.044, grad_fn=<ExpBackward>),
 tensor(1.225, grad_fn=<ExpBackward>),
 tensor(1.731, grad_fn=<ExpBackward>)]

In [389]:
posy = sum(zh)-4
posy

tensor(4.804, grad_fn=<SubBackward0>)

In [390]:
6*torch.rand(4)-3

tensor([-1.072, -0.565, -2.601,  2.317])

In [391]:
nn.Parameter()

Parameter containing:
tensor([], requires_grad=True)

In [412]:
class PosyNet(nn.Module):

    def __init__(self):
        super(PosyNet, self).__init__()
        
        # input: 3-vector (x, y, z)
        self.len_x = 3
        # output: 1-vector, posynomial approximation
        
        # number of monomials: M
        self.M = 9
        
        
        self.w = nn.Parameter(6*torch.rand(self.len_x, self.M)-3)
        #self.powers = torch.FloatTensor([[0, -1, 1, 1, 1],[1, -2, 0, 2, 1], [0, -1, 0, 2, 1]]).t()
        self.b = nn.Parameter(5*torch.rand(self.M))
        
        
    def forward(self, x):
        
        y = torch.log(x)
        
        z = [torch.log(1+torch.exp(torch.matmul(self.w[:,i], y) + self.b[i])) for i in range(self.M)]
        
        zh = [torch.exp(zz) for zz in z]
        
        posy = sum(zh)-self.M
           
        return posy
    


In [413]:
def f(x):

    out = x[1]**1.5*x[2]**3 + 2*x[0]**2*x[2]**-1 + 3*x[1]**3.2 + 4*x[0]**0.5*x[1]**-2*x[2]
    
    return out

In [414]:
torch.norm(x)

tensor(2.412)

In [415]:
torch.rand(1)

tensor([0.879])

In [416]:
net = PosyNet()
print(net)

import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

torch.set_printoptions(precision=3, sci_mode=False)

PosyNet()


In [417]:
list(net.parameters())

[Parameter containing:
 tensor([[-2.846, -0.609, -1.406, -2.838, -2.026, -0.980,  2.593, -1.503, -0.921],
         [-1.149, -1.108, -1.907,  2.137, -2.650,  2.437,  0.123,  1.995,  0.226],
         [-0.360,  1.448, -2.660,  0.811, -0.082,  2.961, -1.240,  2.773,  0.390]],
        requires_grad=True), Parameter containing:
 tensor([3.911, 4.875, 3.333, 4.468, 3.641, 4.055, 2.442, 2.463, 4.062],
        requires_grad=True)]

In [418]:
relative_loss_hist = [100]*5

In [419]:
relative_loss_hist

[100, 100, 100, 100, 100]

In [420]:
sum(relative_loss_hist[1:] + [1])

401

In [421]:
print('starting training')

running_loss = 0
relative_loss = 0


plateau = 10
patience = 10

extra_episodes = 2*patience

relative_loss_hist = [100]*plateau


for epoch in range(1000):
    
    for data in range(600):
    
        x = 3*torch.rand(3) + 0.2

        y = f(x) + torch.normal(mean=0., std=torch.tensor(0.01*50))

        optimizer.zero_grad()
        
        yh = net(x)

        relative_loss += (torch.norm(yh-y)/torch.norm(y)).item()
        loss = (torch.log(yh+1) - torch.log(y+1))**2 + 0.001*torch.norm(net.w, p=1) + 0.001*torch.norm(net.b, p=1)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    
    
    running_loss = running_loss/600
    relative_loss = relative_loss/600
    
    print(f'epoch {epoch} - loss: {running_loss:0.8f}, relative loss: {relative_loss:0.8f}, rel loss history: {(1/plateau)*sum(relative_loss_hist):0.8f}')

    
    if relative_loss > (1/plateau)*sum(relative_loss_hist) and extra_episodes > patience:
        print('plateau met')
        print(relative_loss)
        print((1/plateau)*sum(relative_loss_hist))
        extra_episodes = patience-1
    
    if extra_episodes < patience:
        extra_episodes -= 1
    
    if extra_episodes == 0:
        print('patience met')
        print(relative_loss)
        print((1/plateau)*sum(relative_loss_hist))
        break
    
    relative_loss_hist = relative_loss_hist[1:] + [relative_loss]
    
    running_loss = 0
    relative_loss = 0
    
    if epoch%100 == 0:
        params = list(net.parameters())
        print(params[0])
        print(torch.exp(params[1]))
        
        
params = list(net.parameters())
print(params[0])
print(torch.exp(params[1]))

starting training
epoch 0 - loss: 1.31166174, relative loss: 9.44456946, rel loss history: 100.00000000
Parameter containing:
tensor([[    -0.607,      0.298,     -0.181,     -0.727,     -0.070,     -0.351,
              1.793,      0.031,      0.008],
        [    -0.311,     -1.034,     -0.859,      0.979,     -1.587,      1.212,
              0.601,      2.007,      0.459],
        [    -0.251,      0.668,     -0.642,     -0.085,      0.175,      0.975,
             -0.478,      1.815,     -0.000]], requires_grad=True)
tensor([2.675, 4.789, 2.035, 2.602, 3.133, 2.279, 2.689, 2.192, 4.541],
       grad_fn=<ExpBackward>)
epoch 1 - loss: 0.14108452, relative loss: 0.28645013, rel loss history: 90.94445695
epoch 2 - loss: 0.06908999, relative loss: 0.15881370, rel loss history: 80.97310196
epoch 3 - loss: 0.06043557, relative loss: 0.14261970, rel loss history: 70.98898333
epoch 4 - loss: 0.05826547, relative loss: 0.13762730, rel loss history: 61.00324530
epoch 5 - loss: 0.05588949, re

In [379]:
params[0]

Parameter containing:
tensor([[     0.010,      0.499,      0.001,      1.967],
        [     3.202,     -2.008,      1.470,     -0.000],
        [     0.004,      0.984,      3.010,     -0.981]], requires_grad=True)

In [380]:
torch.exp(params[1])

tensor([2.983, 3.980, 1.007, 2.077], grad_fn=<ExpBackward>)

In [335]:
list(net.parameters())

[Parameter containing:
 tensor([[     0.615,     -0.000,      0.010,      1.975],
         [    -1.787,     -0.000,      2.743,      0.390],
         [     1.178,     -0.001,      0.779,     -1.519]], requires_grad=True),
 Parameter containing:
 tensor([ 1.525, -0.375,  1.565,  0.264], requires_grad=True)]

In [226]:
params = list(net.parameters())

In [241]:
[[float(f'{pp:2.2f}') for pp in p] for p in params[0].data]

[[0.53, -1.81, 0.03, 0.8],
 [-4.64, -2.39, 1.01, 0.66],
 [0.59, -0.0, 0.6, -1.38]]

In [242]:
[float(f'{p:2.2f}') for p in params[1].data]

[3.87, 0.0, 0.7, 0.9]

In [119]:
fvals = []
for _ in range(1000):
    x = 3*torch.rand(3) + 0.2
    y = f(x).item()
    fvals.append(y)

In [120]:
import pandas as pd

df_f = pd.DataFrame(fvals)

In [121]:
df_f.describe()

,0
count,1000.000000
mean,71.740356
std,56.237701
min,3.726056
25%,31.942504
50%,54.288652
75%,97.200777
max,327.032196


In [156]:
torch.normal(mean=0.5, std=torch.tensor(0.01*50))

tensor(-0.2999)

In [244]:
net2 = PosyNet()

In [247]:
list(net2.parameters())

[Parameter containing:
 tensor([[ 2.9530,  1.4717, -1.9047,  1.0967],
         [-0.4320, -0.5977, -1.4674, -1.3579],
         [ 2.9908, -0.8913,  2.5368, -2.0302]], requires_grad=True),
 Parameter containing:
 tensor([4.4431, 1.0680, 1.9927, 4.4015], requires_grad=True)]

In [248]:
net2.w

Parameter containing:
tensor([[ 2.9530,  1.4717, -1.9047,  1.0967],
        [-0.4320, -0.5977, -1.4674, -1.3579],
        [ 2.9908, -0.8913,  2.5368, -2.0302]], requires_grad=True)

In [249]:
net2.b

Parameter containing:
tensor([4.4431, 1.0680, 1.9927, 4.4015], requires_grad=True)

In [256]:
torch.tensor([[0, 2, 0, 0.5], [1.5, 0, 3.2, -2], [3, -1, 0, 1]])


tensor([[ 0.0000,  2.0000,  0.0000,  0.5000],
        [ 1.5000,  0.0000,  3.2000, -2.0000],
        [ 3.0000, -1.0000,  0.0000,  1.0000]])

In [284]:
torch.log([1])

TypeError: log(): argument 'input' (position 1) must be Tensor, not list

In [289]:
t = torch.tensor([1., 2, 3, 4])
t

tensor([1., 2., 3., 4.])

In [290]:
torch.log(t)

tensor([0.0000, 0.6931, 1.0986, 1.3863])

In [291]:
class PosyNet2(nn.Module):

    def __init__(self):
        super(PosyNet2, self).__init__()
        
        # input: 3-vector (x, y, z)
        self.len_x = 3
        # output: 1-vector, posynomial approximation
        
        # number of monomials: M
        self.M = 4
        
        
        self.w = torch.FloatTensor([[0, 2, 0, 0.5], [1.5, 0, 3.2, -2], [3, -1, 0, 1]])
        #self.powers = torch.FloatTensor([[0, -1, 1, 1, 1],[1, -2, 0, 2, 1], [0, -1, 0, 2, 1]]).t()
        self.b = torch.log(torch.tensor([1., 2, 3, 4]))
        
        
    def forward(self, x):
        
        y = torch.log(x)
        
        z = [torch.log(1+torch.exp(torch.matmul(self.w[:,i], y) + self.b[i])) for i in range(self.M)]
        
        zh = [torch.exp(zz) for zz in z]
        
        posy = sum(zh)-self.M
        
        return posy
    



In [292]:
net2 = PosyNet2()

In [293]:
x = 3*torch.rand(3)+0.2
x

tensor([2.7905, 1.9097, 1.6581])

In [294]:
net2(x)

tensor(48.2384)

In [295]:
f(x)

tensor(48.2384)